In [523]:
from pykrx import stock
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import warnings
import requests
import zipfile
import os, json
import math
import xml.etree.ElementTree as ET


from tensorflow.python.ops.gen_boosted_trees_ops import boosted_trees_sparse_calculate_best_feature_split

# 모든 컬럼 출력
pd.set_option('display.max_columns', None)
# 모든 행 출력
pd.set_option('display.max_rows', None)
# 열 너비 무제한(문자열 길어도 줄바꿈 없이 출력)
pd.set_option('display.max_colwidth', None)
#warnings.filterwarnings("ignore", category=FutureWarning)



In [526]:
with open('secrets.json', 'r') as f:
    secrets =json.load(f)
API_KEY = secrets['DART_API_KEY']


'08aebbb7c442c2d40383c1c71a2bb55839daff3b'

### DART기반 1차후출보 추출

In [8]:
# 1. 저장할 폴더 지정
save_dir = "./dart"  # 원하는 폴더
os.makedirs(save_dir, exist_ok=True)

# 2. 다운로드할 ZIP 파일 경로
zip_path = os.path.join(save_dir, "corpCode.zip")

# 3. DART corpCode.zip 다운로드
url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
response = requests.get(url)

# 4. ZIP 파일로 저장
with open(zip_path, "wb") as f:
    f.write(response.content)

# print("다운로드 완료:", zip_path)

# 5. ZIP 압축 해제
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(save_dir)

# 6. 압축 안의 XML 파일명 출력
extracted_files = z.namelist()
# print("압축에서 추출된 파일:", extracted_files)

# 7. 최종 XML 경로
xml_path = os.path.join(save_dir, extracted_files[0])
# print("최종 XML 파일 경로:", xml_path)

In [13]:
# corp_code.xml 파싱
xml_path = os.path.join('./dart', "CORPCODE.xml")
tree = ET.parse(xml_path)
root = tree.getroot()

dict_corp_nm = {}
for c in root.findall('list'):
    dict_corp_nm[c.find('corp_name').text] = c.find('corp_code').text


In [16]:
dict_tickers = {}
dict_tickers_inv = {}
dict_kospi_cd = {}
tickers = stock.get_market_ticker_list(market='KOSPI')
for ticker in tickers:
    ticker_nm = stock.get_market_ticker_name(ticker)
    dict_tickers_inv[ticker_nm] = ticker
    dict_tickers[ticker] = ticker_nm
    if ticker_nm in dict_corp_nm:
        dict_kospi_cd[ticker_nm] = dict_corp_nm[ticker_nm]

In [190]:
def get_fs_quarter(api_key, corp_code, year, quarter, fs_div="CFS"):
    """
    reprt_code mapping:
      1Q -> 11013
      2Q -> 11012
      3Q -> 11014
      FY(사업보고서) -> 11011 (연말)
    fs_div: "CFS" or "OFS"
    """
    quarter_map = {1: "11013", 2: "11012", 3: "11014", 4: "11011"}
    reprt_code = quarter_map[quarter]
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": api_key,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": reprt_code,
        "fs_div": fs_div
    }
    r = requests.get(url, params=params).json()
    if r.get("status") == "000":
        return pd.DataFrame(r.get("list", []))
    else:
        # status가 013이면 데이터 없음 등
        if r.get("status") == "013":
            return pd.DataFrame()
        else:
            print("API status:", r.get("status"), r.get("message"))
        return pd.DataFrame()

In [534]:
TAG_MAP = {
    # 손익계산서
    "revenue": [
        "ifrs-full_Revenue",              # IFRS 매출
        "dart_Revenue",                   # DART 매출
        "ifrs-full_SalesRevenue",         # IFRS 매출(다른 표기)
        "ifrs-full_GrossSales"            # 총매출
    ],
    "cogs": [
        "ifrs-full_CostOfSales",          # 매출원가
        "dart_CostofGoodsSold"            # DART 매출원가
    ],
    "operating_income": [
        "dart_OperatingIncomeLoss",       # 영업이익
        "ifrs-full_OperatingProfitLoss"   # IFRS 영업이익
    ],
    "net_income": [
        "ifrs-full_ProfitLoss",                               # 당기순이익
        "ifrs-full_ProfitLossAttributableToOwnersOfParent",  # 지배주주 기준 순이익
        "dart_NetIncomeLoss"                                  # DART 순이익
    ],
    "net_owners": [              #지배부분
        "ifrs-full_ProfitLossAttributableToOwnersOfParent"
    ],

    # 재무상태표
    "total_assets": [
        "ifrs-full_Assets",   # 자산총계
        "dart_Assets"
    ],
    "cash_and_cash_equivalents": [
        "ifrs-full_CashAndCashEquivalents",  # 현금 및 현금성자산
        "dart_CashAndCashEquivalents"
    ],
    "accounts_receivable": [
        "ifrs-full_TradeAndOtherReceivables", # 매출채권
        "dart_AccountsReceivable"
    ],
    "current_assets": [
        "ifrs-full_CurrentAssets",  # 유동자산
        "dart_CurrentAssets"
    ],
    "inventories": [
        "ifrs-full_Inventories",    # 재고자산
        "dart_Inventories"
    ],
    "non_current_assets": [
        "ifrs-full_NonCurrentAssets", # 비유동자산
        "dart_NonCurrentAssets"
    ],
    "current_liabilities": [
        "ifrs-full_CurrentLiabilities",  # 유동부채
        "dart_CurrentLiabilities"
    ],
    "non_current_liabilities": [
        "ifrs-full_NonCurrentLiabilities", # 비유동부채
        "dart_NonCurrentLiabilities"
    ],
    "total_liabilities": [
        "ifrs-full_Liabilities",   # 부채총계
        "dart_Liabilities"
    ],
    "total_equity": [
        "ifrs-full_Equity",        # 자본총계
        "dart_Equity"
    ],
    "equity_parent": [
        "ifrs-full_EquityAttributableToOwnersOfParent",  # 지배지분
    ],

    # ## 배당금 여부
    # 'dividend_cash': [
    #     'ifrs-full_DividendsDeclaredAndPayable',
    #     'ifrs-full_DividendsPaid',
    #     'dart_DividendsDeclared' #배당금 총액(현금)
    # ],
    # 'dividend_per_share': [
    #     'ifrs-full_CashDividendsPerShare', 'dart_DividendsPerShare' #주당 배당금(DPS)
    # ]
}


In [543]:
# dict_kospi_cd
def get_dart_data(api_key, dict_cd, year, quarter, fs_div="CFS"):
    dict_corp_result = {}
    for corp_nm in dict_cd.keys():
        df_q = get_fs_quarter(api_key, dict_cd[corp_nm], year, quarter, fs_div="CFS")
        if df_q.empty:
            continue

        df_cls = df_q[df_q.sj_div == 'CIS'] ## 포괄손익계산서
        df_ls = df_q[df_q.sj_div == 'IS']   ## 손익계산서
        df_bs = df_q[df_q.sj_div == 'BS']   ## 재무상태표
        #df_sce = df_q[df_q.sj_div == 'SCE'] ## 자본변동표

        # CIS(포괄손익계산서) 우선, 없으면 IS(손익계산서)
        df_combined = pd.concat([df_cls, df_ls, df_bs])

        dict_corp_result[corp_nm] = {}
        for tag in TAG_MAP.keys():
            is_existed_row = False
            for a_id in TAG_MAP[tag]:
                row = df_combined[df_combined['account_id'] == a_id]
                if not row.empty:
                    if quarter == 4:
                        dict_corp_result[corp_nm][tag] = {
                            'thstrm_amount' :
                                row.iloc[0]['thstrm_amount'] if 'thstrm_amount' in row.columns else None,
                            'frmtrm_amount' :
                                row.iloc[0]['frmtrm_amount'] if 'frmtrm_amount' in row.columns else None,
                            'bfefrmtrm_amount' :
                                row.iloc[0]['bfefrmtrm_amount'] if 'bfefrmtrm_amount' in row.columns else None
                        }
                    else:
                        dict_corp_result[corp_nm][tag] = {
                            'thstrm_amount' :
                                row.iloc[0]['thstrm_amount'] if 'thstrm_amount' in row.columns else None
                        }
                    is_existed_row = True
                    break
            if not is_existed_row:
                dict_corp_result[corp_nm][tag] = None
    return dict_corp_result


def extract_value_from_tag(dict_corp_result, tag, year, report_period='yearly'):
    dict_extract_value = {'corp_nm':[], f'{tag}_{year}':[]}
    if report_period == 'yearly':
        dict_extract_value = {'corp_nm':[], f'thstrm_{tag}':[], f'frmtrm_{tag}':[], f'bfefrmtrm_{tag}':[]}

    for corp_nm in dict_corp_result.keys():
        dict_value = dict_corp_result[corp_nm][tag]

        if dict_value is None: continue
        if any(v is None or (isinstance(v, float) and math.isnan(v)) for v in dict_value.values()): continue
        if any(v == '' for v in dict_value.values()): continue

        thstrm_amount = int(dict_value['thstrm_amount']) // 100000000 #억 단위
        if report_period == 'yearly':
            frmtrm_amount = int(dict_value['frmtrm_amount'])
            bfefrmtrm_amount = int(dict_value['bfefrmtrm_amount'])

        dict_extract_value['corp_nm'].append(corp_nm)
        dict_extract_value[f'{tag}_{year}'].append(thstrm_amount)
        if report_period == 'yearly':
            dict_extract_value[f'thstrm_{tag}'].append(thstrm_amount)
            dict_extract_value[f'frmtrm_{tag}'].append(frmtrm_amount)
            dict_extract_value[f'bfefrmtrm_{tag}'].append(bfefrmtrm_amount)

    df_extracted = pd.DataFrame(dict_extract_value)
    return df_extracted


# def calc_growth(df, tag):
#     df[f'{tag}_growth1'] = np.round((df[f'thstrm_{tag}'] - df[f'frmtrm_{tag}']) / df[f'frmtrm_{tag}'], 2 )
#     df[f'{tag}_growth2'] = np.round((df[f'frmtrm_{tag}'] - df[f'bfefrmtrm_{tag}']) / df[f'bfefrmtrm_{tag}'], 2 )
#     return df
#
#
# def calc_margin(df, tag):
#     df[f'{tag}_margin1'] = np.round(df[f'thstrm_{tag}'] / df[f'thstrm_revenue'], 2 )
#     df[f'{tag}_margin2'] = np.round(df[f'frmtrm_{tag}'] / df[f'frmtrm_revenue'], 2 )
#     df[f'{tag}_margin3'] = np.round(df[f'bfefrmtrm_{tag}'] / df[f'bfefrmtrm_revenue'], 2 )
#     return df


def calc_safe_growth(curr, prev):
    if pd.isna(curr) or pd.isna(prev) or prev == 0 or curr == 'N/A' or prev == 'N/A':
        return np.nan
    return np.round((curr - prev) / prev * 100, 2)


def calc_safe_margin(curr, prev):
    if pd.isna(curr) or pd.isna(prev) or prev == 0 or curr == 'N/A' or prev == 'N/A':
        return np.nan
    return np.round(curr / prev * 100, 2)


In [544]:
quarter = 3
list_year = [2023, 2024, 2025]
df_report = pd.DataFrame()
for year in list_year:
    dict_corp_result = get_dart_data(API_KEY, dict_kospi_cd, year, quarter)
    for tag in TAG_MAP.keys():
        df_extracted = extract_value_from_tag(dict_corp_result, tag, year, 'quarter')
        if df_report.empty:
            df_report = df_extracted
        else:
            df_report = df_report.merge(df_extracted, on='corp_nm', how='outer')


df_report = df_report.fillna('N/A')

In [498]:

col_revenue = [ col for col in df_report.columns if col.startswith('revenue')]
col_cogs = [col for col in df_report.columns if col.startswith('cogs')]
col_operating_income = [col for col in df_report.columns if col.startswith('operating_income')]
col_net_income = [col for col in df_report.columns if col.startswith('net_income')]
col_net_owners = [col for col in df_report.columns if col.startswith('net_owners')]
col_equity_parent = [col for col in df_report.columns if col.startswith('equity_parent')]

col_total_liabilities = [col for col in df_report.columns if col.startswith('total_liabilities')]
col_total_equity = [col for col in df_report.columns if col.startswith('total_equity')]
col_current_liabilities = [col for col in df_report.columns if col.startswith('current_liabilities')]
col_current_assets = [col for col in df_report.columns if col.startswith('current_assets')]

### ROE를 위한 전처리
### 당기순이익_지배주주 / (자본총계 - 비지배지분)

# 조건 정의
cond_revenue_na   = df_report[col_revenue[-1]] == 'N/A'
cond_oper_na      = df_report[col_operating_income[-1]] == 'N/A'
cond_net_na       = df_report[col_net_owners[-1]] == 'N/A'

# 정상 보고서, 매출 없음, 영업이익 없음(매출있음), 순이익 없음
df_normal = df_report[~cond_revenue_na & ~cond_oper_na & ~cond_net_na].copy()
df_no_revenue = df_report[cond_revenue_na].copy() ## 순이익만
df_no_oper = df_report[~cond_revenue_na & cond_oper_na].copy() ##매출, 순이익 있음
df_no_net = df_report[cond_net_na].copy() ## pass

### calc revenue growth
for idx, (v1, v2) in enumerate(zip(col_revenue, col_revenue[1:])):
    df_normal.loc[:, f'revenue_growth{idx}'] = df_normal.apply(lambda x: calc_safe_growth(x[v2], x[v1]), axis=1)
    df_no_oper.loc[:, f'revenue_growth{idx}'] = df_no_oper.apply(lambda x: calc_safe_growth(x[v2], x[v1]), axis=1)

### calc net_income growth
for idx, (v1, v2) in enumerate(zip(col_revenue, col_revenue[1:])):
    df_no_revenue.loc[:, f'net_growth{idx}'] = df_no_revenue.apply(lambda x: calc_safe_growth(x[v2], x[v1]), axis=1)

### calc cogs margin
for idx, (v1, v2) in enumerate(zip(col_cogs, col_revenue)):
    df_normal.loc[:, f'cogs_margin{idx}'] = df_normal.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)

### calc operating margin
for idx, (v1, v2) in enumerate(zip(col_operating_income, col_revenue)):
    df_normal.loc[:, f'oper_margin{idx}'] = df_normal.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)

### calc net margin
for idx, (v1, v2) in enumerate(zip(col_net_owners, col_revenue)):
    df_normal.loc[:, f'net_margin{idx}'] = df_normal.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)
    df_no_oper.loc[:, f'net_margin{idx}'] = df_no_oper.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)

### calc equity margin
for idx, (v1, v2) in enumerate(zip(col_net_owners, col_equity_parent)):
    df_normal.loc[:, f'equity_margin{idx}'] = df_normal.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)
    df_no_oper.loc[:, f'equity_margin{idx}'] = df_no_oper.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)
    df_no_revenue.loc[:, f'equity_margin{idx}'] = df_no_revenue.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)

### calc liabilities margin
for idx, (v1, v2) in enumerate(zip(col_total_liabilities, col_equity_parent)):
    df_normal.loc[:, f'liabilities_margin{idx}'] = df_normal.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)
    df_no_oper.loc[:, f'liabilities_margin{idx}'] = df_no_oper.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)
    df_no_revenue.loc[:, f'liabilities_margin{idx}'] = df_no_revenue.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)


### calc current_liabilities margin
for idx, (v1, v2) in enumerate(zip(col_current_liabilities, col_current_assets)):
    df_normal.loc[:, f'current_liabilities_margin{idx}'] = df_normal.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)
    df_no_oper.loc[:, f'current_liabilities_margin{idx}'] = df_no_oper.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)
    df_no_revenue.loc[:, f'current_liabilities_margin{idx}'] = df_no_revenue.apply(lambda x: calc_safe_margin(x[v1], x[v2]), axis=1)



In [506]:
### 조건 (df_normal)
## 1) 3년 연속 매출액 및 매출액 대비 당기순이익 증가
## 2) 최근 3년동안 매출액 대비 당기순이익율 마이너스 없음
## 3) 최근 3년동안 유동부채 대비 유동자산 50% 이하
## 4) 최근 3년동안 자본총액 대비 부채총액 100% 이하



df_normal.loc[:, 'count_net_margin_over_0'] = ((df_normal[['net_margin0','net_margin1','net_margin2']] > 0).sum(axis=1))
df_normal.loc[:, 'count_liabilities_margin_below_100'] = ((df_normal[['liabilities_margin0','liabilities_margin1','liabilities_margin2']] < 100).sum(axis=1))
df_normal.loc[:, 'count_current_liabilities_margin_over_50'] = ((df_normal[['current_liabilities_margin0','current_liabilities_margin1','current_liabilities_margin2']] >= 50).sum(axis=1))

cond_revenue_growth = df_normal['revenue_growth0'] < df_normal['revenue_growth1']
count_liabilities_margin_below_100 = df_normal['count_liabilities_margin_below_100'] == 3
cond_current_liabilities = df_normal['count_current_liabilities_margin_over_50'] == 3
cond_count_net_margin_over_0 = df_normal['count_net_margin_over_0'] == 3
cond_net_margin = (df_normal['net_margin0'] < df_normal['net_margin1']) & (df_normal['net_margin1'] < df_normal['net_margin2'])



df_normal_filtered = df_normal[cond_revenue_growth & count_liabilities_margin_below_100 & cond_current_liabilities & cond_count_net_margin_over_0 & cond_net_margin ]
df_normal_filtered[["corp_nm"] + col_revenue + col_operating_income + col_net_income ]

,corp_nm,revenue_2023,revenue_2024,revenue_2025,operating_income_2023,operating_income_2024,operating_income_2025,net_income_2023,net_income_2024,net_income_2025
98,NAVER,24452.0,27155.0,31380.0,3801.0,5252.0,5706.0,3562.0,5300.0,7347.0
228,대한제분,3604.0,3294.0,3476.0,142.0,172.0,165.0,108.0,154.0,228.0
235,더존비즈온,845.0,970.0,1146.0,149.0,200.0,348.0,76.0,106.0,234.0
298,미원홀딩스,1112.0,1174.0,1282.0,42.0,47.0,69.0,39.0,56.0,90.0
604,한솔케미칼,1882.0,1906.0,2300.0,355.0,370.0,488.0,302.0,322.0,464.0


In [508]:
### 조건(df_no_oper)
## 1) 3년 자기자본 대비 부채비율 증가 No
## 2) 3년 자기자본 대비 당기순이익률 증가
cond_liabilities = (df_no_oper['liabilities_margin0'] < df_no_oper['liabilities_margin1']) & (df_no_oper['liabilities_margin1'] < df_no_oper['liabilities_margin2'])
cond_net_over_equity1 = df_no_oper['equity_margin0'] < df_no_oper['equity_margin1']
cond_net_over_equity2 = df_no_oper['equity_margin1'] < df_no_oper['equity_margin2']

df_no_oper_filtered = df_no_oper[~cond_liabilities & cond_net_over_equity1 & cond_net_over_equity2 ]
df_no_oper_filtered[["corp_nm"] + col_net_income + ['liabilities_margin0', 'liabilities_margin1', 'liabilities_margin2']
                    + ['equity_margin0'] + ['equity_margin1'] + ['equity_margin2']]

,corp_nm,net_income_2023,net_income_2024,net_income_2025,liabilities_margin0,liabilities_margin1,liabilities_margin2,equity_margin0,equity_margin1,equity_margin2
14,DB증권,76.0,59.0,355.0,969.13,924.49,1051.53,0.63,0.64,2.95
578,한국금융지주,2121.0,3145.0,6748.0,1044.56,1000.44,961.41,2.52,3.37,5.81


In [507]:
### 조건(df_no_revenue)
## 1) 3년 자기자본 대비 부채비율 증가 No
## 2) 3년 자기자본 대비 당기순이익률 증가
## 중요한 이유! ==> 내 돈을 가지고 얼마나 효율적으로 굴리는지 보여줌
## 자기자본 = 주주가 회사에 넣어둔 원금 같은 것, 그리고 공장도 재고도 없기에 자본이 곧 생산설비를 의미함
cond_liabilities = ((df_no_revenue['liabilities_margin0'] < df_no_revenue['liabilities_margin1']) &
                    (df_no_revenue['liabilities_margin1'] < df_no_revenue['liabilities_margin2']))
cond_net_over_equity1 = df_no_revenue['equity_margin0'] < df_no_revenue['equity_margin1']
cond_net_over_equity2 = df_no_revenue['equity_margin1'] < df_no_revenue['equity_margin2']
df_no_revenue_filtered = df_no_revenue[~cond_liabilities & cond_net_over_equity1 & cond_net_over_equity2 ]
df_no_revenue_filtered[['corp_nm'] + col_net_income + col_net_owners + col_equity_parent +
                       ['liabilities_margin0', 'liabilities_margin1', 'liabilities_margin2'] +
                       ['equity_margin0'] + ['equity_margin1'] + ['equity_margin2']]

,corp_nm,net_income_2023,net_income_2024,net_income_2025,net_owners_2023,net_owners_2024,net_owners_2025,equity_parent_2023,equity_parent_2024,equity_parent_2025,liabilities_margin0,liabilities_margin1,liabilities_margin2,equity_margin0,equity_margin1,equity_margin2
649,현대차증권,93.0,106.0,122.0,93.0,106.0,122.0,12725.0,12994.0,14375.0,884.65,863.18,810.6,0.73,0.82,0.85


In [509]:
list_target_corp_nm = df_normal_filtered['corp_nm'].unique().tolist() + \
                        df_no_oper_filtered['corp_nm'].unique().tolist() + \
                        df_no_revenue_filtered['corp_nm'].unique().tolist()

list_target_corp_nm

['NAVER', '대한제분', '더존비즈온', '미원홀딩스', '한솔케미칼', 'DB증권', '한국금융지주', '현대차증권']

### KRX기반 최종후출보 추출

In [516]:
dict_tickers = {}
dict_tickers_inv = {}
tickers = stock.get_market_ticker_list(market='KOSPI')
for ticker in tickers:
    ticker_nm = stock.get_market_ticker_name(ticker)
    if type(ticker_nm) == str:
        dict_tickers_inv[ticker_nm] = ticker
        dict_tickers[ticker] = ticker_nm

In [518]:
list_tickers = [ dict_tickers_inv[corp_nm] for corp_nm in list_target_corp_nm]
list_tickers

['035420',
 '001130',
 '012510',
 '107590',
 '014680',
 '016610',
 '071050',
 '001500']

In [519]:

def get_krx_data(list_tickers):
    # ================================================
    # 1. 오늘 날짜 설정
    # ================================================
    dt = datetime.now().strftime("%Y%m%d")
    #list_dt.append(today)

    # 결과 저장
    rows = []
    # for dt in list_dt:
    #     # ================================================
    #     # 2. KOSPI 종목 전체
    #     # ================================================
    #     tickers = stock.get_market_ticker_list(dt, market="KOSPI")

    # ================================================
    # 3. Daily 데이터 로딩
    # ================================================
    fund = stock.get_market_fundamental(dt)     # EPS, PER, DIV, BPS 등
    price_df = stock.get_market_ohlcv(dt)       # 종가
    mcap_df = stock.get_market_cap(dt)          # 시가총액

    # 결과 저장
    # rows = []

    for ticker in list_tickers:
        try:
            name = stock.get_market_ticker_name(ticker)
            price = price_df.loc[ticker]["종가"] # 주가
            mcap = mcap_df.loc[ticker]["시가총액"]
            volume = mcap_df.loc[ticker]["거래량"]
            trading_value = mcap_df.loc[ticker]["거래대금"]
            #shares_outstanding = mcap_df.loc[ticker]["상장주식수"]
            bps = fund.loc[ticker]['BPS']  # 자기자본(=자산 - 부채) / 방행주식수
            per = fund.loc[ticker]['PER']
            pbr = fund.loc[ticker]['PBR']
            div = fund.loc[ticker]["DIV"]  # 배당수익률 %
            eps = fund.loc[ticker]["EPS"]
            dps = fund.loc[ticker]['DPS']  # 주당 배당금


            # ===============================
            # 기본 값
            # ===============================
            #shares = mcap / price  if price                   # 발행주식수
            if price in [0, None] or pd.isna(price) or mcap == 0 or pd.isna(mcap):
                shares = np.nan
            else:
                shares = mcap / price
            dps = price * (div / 100)                # 주당 배당금
            total_div = mcap * (div / 100)           # 총배당금
            net_income = eps * shares                # 연간 순이익(TTM)
            if per in [0, None] or pd.isna(per) or pbr == 0 or pd.isna(pbr):
                roe = np.nan
            else:
                roe = pbr / per

            # ===============================
            # 연간 총주주환원율
            # ===============================
            srr_year = total_div / net_income if net_income > 0 else None


            rows.append({
                "dt": dt,
                "yy": dt[:4],
                "티커": ticker,
                "종목명": name,
                "주가": price,
                "BPS": bps,
                "PER": per,
                'PBR': pbr,
                'ROE': np.round(roe, 2),
                'DIV': div,
                'EPS': eps,
                'DPS': dps,
                '시가총액': mcap,
                '거래량': volume,
                '거래대금': trading_value,
                # 연간 값
                "순이익": round(net_income, 0),
                "총배당금": round(total_div, 0),
                "주식수":shares,
                "주주환원율": round(srr_year, 4) if srr_year else None,

            })
        except:
            continue

    # ================================================
    # 4. DataFrame으로 변환
    # ================================================
    df = pd.DataFrame(rows)
    return df


df_final = get_krx_data(list_tickers)

In [520]:
df_final

,dt,yy,티커,종목명,주가,BPS,PER,PBR,ROE,DIV,EPS,DPS,시가총액,거래량,거래대금,순이익,총배당금,주식수,주주환원율
0,20251124,2025,035420,NAVER,266750.0,170953.0,21.02,1.56,0.07,0.42,12702.0,1120.350,41840441186500,470562,123773854000,1.992342e+12,1.757299e+11,156852638.0,0.0882
1,20251124,2025,001130,대한제분,140400.0,628059.0,4.80,0.22,0.05,2.49,29257.0,3495.960,237276000000,384,53912800,4.944433e+10,5.908172e+09,1690000.0,0.1195
2,20251124,2025,012510,더존비즈온,92900.0,18502.0,42.81,5.02,0.12,0.51,2170.0,473.790,2822560633600,31206,2876676250,6.593064e+10,1.439506e+10,30382784.0,0.2183
3,20251124,2025,107590,미원홀딩스,67400.0,112416.0,6.18,0.60,0.10,1.04,10906.0,700.960,156368000000,14,944000,2.530192e+10,1.626227e+09,2320000.0,0.0643
4,20251124,2025,014680,한솔케미칼,215500.0,88053.0,19.63,2.45,0.12,0.97,10980.0,2090.350,2442734522500,16755,3531884750,1.244604e+11,2.369452e+10,11335195.0,0.1904
5,20251124,2025,016610,DB증권,10240.0,25569.0,8.30,0.40,0.05,3.91,1233.0,400.384,434651023360,64079,646530665,5.233640e+10,1.699486e+10,42446389.0,0.3247
6,20251124,2025,071050,한국금융지주,158100.0,165262.0,8.88,0.96,0.11,2.52,17799.0,3984.120,8810279335200,81076,12523895750,9.918669e+11,2.220190e+11,55725992.0,0.2238
7,20251124,2025,001500,현대차증권,8080.0,30047.0,8.63,0.27,0.03,2.23,936.0,180.184,499610995520,23004,185463010,5.787573e+10,1.114133e+10,61833044.0,0.1925


In [522]:
df_final[(df_final['PER'] < 10) & (df_final['PBR'] < 1) ].sort_values(by=['PER', 'PBR','주주환원율'], ascending=[True, True, False])

,dt,yy,티커,종목명,주가,BPS,PER,PBR,ROE,DIV,EPS,DPS,시가총액,거래량,거래대금,순이익,총배당금,주식수,주주환원율
1,20251124,2025,001130,대한제분,140400.0,628059.0,4.80,0.22,0.05,2.49,29257.0,3495.960,237276000000,384,53912800,4.944433e+10,5.908172e+09,1690000.0,0.1195
3,20251124,2025,107590,미원홀딩스,67400.0,112416.0,6.18,0.60,0.10,1.04,10906.0,700.960,156368000000,14,944000,2.530192e+10,1.626227e+09,2320000.0,0.0643
5,20251124,2025,016610,DB증권,10240.0,25569.0,8.30,0.40,0.05,3.91,1233.0,400.384,434651023360,64079,646530665,5.233640e+10,1.699486e+10,42446389.0,0.3247
7,20251124,2025,001500,현대차증권,8080.0,30047.0,8.63,0.27,0.03,2.23,936.0,180.184,499610995520,23004,185463010,5.787573e+10,1.114133e+10,61833044.0,0.1925
6,20251124,2025,071050,한국금융지주,158100.0,165262.0,8.88,0.96,0.11,2.52,17799.0,3984.120,8810279335200,81076,12523895750,9.918669e+11,2.220190e+11,55725992.0,0.2238
